<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex12notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex12notebookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


---
## 主成分分析に関する実習
---

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

---
### 計算問題

ほぼ線形代数の復習...

#### 問1

$\mathbf{w}_1 = (1, 0, 1), \mathbf{x}_1 = (1, 2, 3)$ のとき，次の値を求めなさい．
1. $\mathbf{w}_1\cdot\mathbf{x}_1$
1. $\Vert\mathbf{w}_1\Vert$ および $\Vert\mathbf{x}_1\Vert$

次のセルを実行すると，答えを確認できます（数値的に計算した結果なので誤差を含みますが）．

以下では， $\Vert\mathbf{x}_1\Vert^2 = \mathbf{x}_1 \cdot \mathbf{x}_1 $ という性質を利用して $\Vert\mathbf{x}_1\Vert$ を計算してます．

In [ ]:
w1 = np.array([1.0, 0.0, 1.0])
x1 = np.array([1.0, 2.0, 3.0])
print('w1 = ', w1)
print('x1 = ', x1)
print('w1 と x1 の内積 = ', w1 @ x1)
print('|w1| = ', np.sqrt(w1 @ w1))
print('|x1| = ', np.sqrt(x1 @ x1))

#### 問2

$\mathbf{w}_2 = (0, 1, -1), \mathbf{x}_2 = (4, 5, 6)$ のとき，次の値を求めなさい．$\mathbf{w}_1, \mathbf{x}_1$ は問1のものを使ってください．以下，前の問題で出てきた記号を説明せずに使ってる場合があります．
1. $\mathbf{w}_1\cdot\mathbf{x}_2$
1. $\mathbf{w}_2\cdot\mathbf{x}_1$
1. $\mathbf{w}_2\cdot\mathbf{x}_2$


In [ ]:
w2 = np.array([0.0, 1.0, -1.0])
x2 = np.array([4.0, 5.0, 6.0])
print('w2 = ', w2)
print('x2 = ', x2)
print('w1 と x2 の内積 = ', w1 @ x2)
print('w2 と x1 の内積 = ', w2 @ x1)
print('w2 と x2 の内積 = ', w2 @ x2)

#### 問3

次のセルを実行すると表示されるような，3行4列の行列を $X$ とします．
この行列を列ごとに眺めると，第1列は $\mathbf{x}_1$ と等しく，第2列は $\mathbf{x}_2$ と等しいことに注意しましょう．

In [ ]:
X = np.arange(1, 13).reshape((4, 3)).T
X

また，次のような 2行3列の行列を $W$ とします．こちらは，行ごとに眺めると，第1行が $\mathbf{w}_1$ と等しく，第2行が $\mathbf{w}_2$ と等しくなっています．

In [ ]:
W = np.vstack((w1, w2))
W

このとき，$Y = WX$ を手計算で求めなさい．

次のセルを実行すると，答えを確認できます．

In [ ]:
Y = W @ X
Y

結果を見ると，$Y$ の1列目には，$\mathbf{w}_1\cdot\mathbf{x}_1$ と $\mathbf{w}_2\cdot\mathbf{x}_1$ の値が入っており，2列目には，$\mathbf{w}_1\cdot\mathbf{x}_2$ と $\mathbf{w}_2\cdot\mathbf{x}_2$ の値が入っていることが分かります．他の列についても，同様のことが成り立っています．

この例では，$X$ は $3\times 4$ で，3次元ベクトルを4つならべたものと考えられます．このとき，$Y$ は $2\times 4$ の行列となっており，その各列は，$X$の1つの列を作っている 3 次元ベクトルと2つの3次元ベクトル $\mathbf{w}_1, \mathbf{w}_2$ との内積の値を表す2次元ベクトルとなっています．
つまり，$Y$ は，行列 $X$ が表す3次元のデータ4つを，2つの3次元ベクトル $\mathbf{w}_1, \mathbf{w}_2$ を使って2次元に次元削減したものとみなせます．

上記の例では，$X$, $Y$ ともに，データが列ベクトルの形で表されています．線形代数の教科書等で線形変換（一次変換）の話が出てくるときはほとんどこの形でしょう．しかし，コンピュータでデータを扱う場合は，一つのデータが1行で表される形の方が便利です．そういう場合は，行列を転置して考えれば ok です．

In [ ]:
Xt = X.T
Xt

In [ ]:
Yt = Xt @ W.T
Yt

---
### 数物情3次元データの主成分分析

「数学」「物理」「情報」の点数のデータに主成分分析を適用してみましょう（notebookBで使ったのと同じデータです）．

In [ ]:
# 数物情データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/mpi100-mac.csv
dfMPI = pd.read_csv('mpi100-mac.csv', index_col=0)
datMPI = dfMPI.to_numpy().astype(float)
dfMPI

平均を引いたデータを作ります．

In [ ]:
# 平均
Xm = np.mean(dfMPI.to_numpy(), axis=0)
print('平均:', Xm)

# 平均を引いて NumPy 配列にする
X = dfMPI.to_numpy() - Xm
N, D = X.shape
print('X.shape:', X.shape) # X は 100 x 3
print(X[:5, :]) # 最初の5人分を表示

以下のセルを実行すると，`X` の分散共分散行列の固有値と固有ベクトルが求まります．
ここでは，特異値分解という手法を使ってそれらを求めていますが，説明は省きます．

In [ ]:
# 分散共分散行列の固有値と固有ベクトルを求める（Xの特異値分解経由で）
_, sval, Vt = np.linalg.svd(X, full_matrices=False)
eval = sval**2/N
U = Vt
for d in range(D):
    print(f'{d+1}番目の固有値:{eval[d]:.2f}   固有ベクトル:', U[d, :])

得られた3つの固有ベクトルのうち，対応する固有値の大きい2つ（1番目と2番目）を使って行列 `W` を構成し，これを用いて3次元の `X` を 2次元の `Y` に変換します．

In [ ]:
W = U[:2, :]
print(W.shape)
print(W)
Y = X @ W.T # y = Wx の計算
print(Y.shape)
print(Y[:5, :]) # 最初の5人分を表示

`Y` の散布図を描いてみましょう．

In [ ]:
nList = [44, 47, 41, 6] # 特徴的な4人

fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.scatter(Y[:, 0], Y[:, 1])
ax.scatter(Y[nList, 0], Y[nList, 1])
ax.axvline(0, linestyle='-', color='gray')
ax.axhline(0, linestyle='-', color='gray')
ax.set_xlim(-40, 40)
ax.set_ylim(-40, 40)
ax.set_aspect('equal')
ax.set_xlabel('$y_1$')
ax.set_ylabel('$y_2$')
#ax.legend()
for n in nList:
    plt.annotate(f'{n}', (Y[n, 0]+2, Y[n, 1]+2))
plt.show()

図にオレンジ色の点として描かれている4人について，元の点数，Xの値，Y の値を表示すると次のようになります．

In [ ]:
nList = [44, 47, 41, 6]
for n in nList:
    print(f'{n:2d} {X[n]+Xm} {X[n]} {Y[n]}')

固有値最大の固有ベクトルに対応する $\mathbf{w}_1$ は

In [ ]:
W[0, :]

ですので， 3科目の点数からそれぞれの平均を引いた値を並べたベクトル $\mathbf{x} = (\mbox{数学}, \mbox{物理}, \mbox{情報})$ に対して， $y_1 = \mathbf{w}_1\cdot \mathbf{x}$ は
おおよそ次のような式となっています．

$$
y_1 = 0.5 (\mbox{数学}) - 0.5(\mbox{物理}) - 0.7(\mbox{情報})
$$

したがって，「数学が高得点で他の2科目が低得点」なひとは $y_1$ が正となり，「数学が低得点で他の2科目が高得点」なひとは $y_1$ が負となります．上記の4人の点数でそのことを確認してみましょう．

一方，2番目のベクトル $\mathbf{w}_2$ は

In [ ]:
W[1, :]

ですので，$y_2 = \mathbf{w}_2\cdot\mathbf{x}$ はおおよそ次のような式となっています．

$$
y_2 = -0.3 (\mbox{数学}) - 0.9(\mbox{物理}) + 0.4(\mbox{情報})
$$

「情報が高得点で他の2科目が低得点」なひとは  $y_2$  が正となり，「情報が低得点で他の2科目が高得点」なひとは  $y_2$  が負となります．こちらも上記の4人の点数でそのことを確認してみましょう．

---
### 外食への年間支出金額データの主成分分析

独立行政法人統計センターが作成している [教育用標準データセット(SSDSE)](https://www.nstac.go.jp/use/literacy/ssdse/) の中の「SSDSE-家計消費」というデータを使って，主成分分析の実験をやってみましょう．

「SSDSE-家計消費」は，47都道府県の県庁所在市で調査された家計消費に関する226項目の数値から成るデータです．「二人以上の世帯の1世帯あたりの焼き肉（外食）への年間支出金額」みたいな数値が含まれています．

データの詳細はこちら: https://www.nstac.go.jp/sys/files//kaisetsu-C-2022.pdf

この notebook の作成にあたっては，データの前処理の部分でこちらを参考にさせていただきました： https://studiolab.sagemaker.aws/import/github/icoxfog417/mlnote-note/blob/main/notebooks/chapter10_answer.ipynb

#### データの入手と前処理

SSDSEのサイトから CSV ファイルを入手して，扱いやすいように加工します．



In [ ]:
URL = 'https://www.nstac.go.jp/sys/files//SSDSE-C-2022.csv'
df_raw = pd.read_csv(URL, encoding='cp932') # 文字コードが UTF8 ではなく ShiftJIS なので，encoding オプションを指定して変換

# df_raw の見出し行に記されたコードと，1行目に記されたその項目名との対応表を作り，項目名の行を削除
code = df_raw.iloc[0, :]
print('##### コードと項目名の対応')
print(code)
df = df_raw.drop(0, axis=0)

# 'SSDSE-C-2022'列の都市コードと'Prefecture'列の都道府県名，'City'列の県庁所在地名を整理
city = df.iloc[:, :3]
print()
print('##### 都市コード，都道府県名，県庁所在地名')
print(city.head(5))
df = df.drop(['Prefecture', 'City'], axis=1)
df = df.set_index('SSDSE-C-2022')

df = df.drop('R00000', axis=0) # 'R00000'列は「全国」のデータなのでこれを削除
df = df.astype('float32') # 値を浮動小数点型に変換

print()
df.head(5)

ここでは，各種の「外食」での支出金額のみを特徴として取り出します．13個の特徴量があります．

In [ ]:
# 一人あたりの外食支出金額のデータを作成
df_feature = pd.DataFrame()

# コードが 'LB12' で始まる列が，様々な品目の外食支出額．ただし，1世帯あたりの金額
columns = {}
for c in df.columns:
    if c.startswith('LB12') and c != 'LB12':
        print(c, code[c])
        columns[c] = code[c]
        df_feature[c] = df[c]

# 列名をコードから項目名に変更
df_feature = df_feature.rename(columns=columns)

# 都道府県ごとの平均世帯人数で割って一人あたりの金額にする
df_feature = df_feature.divide(df['LA03'], axis=0)

# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 1

df_feature.head(5)

---
#### 主成分分析して可視化してみよう



平均を 0 にしたデータ行列 X を作ります．データの次元数 D = 13, データ数 N = 47 です．

In [ ]:
# 平均を 0 にしたデータ行列 X を作る
X = df_feature.to_numpy()
Xm = np.mean(X, axis=0)
X -= Xm
N, D = X.shape
print(X.shape)
print(X[:5, :]) # 最初の5件の特徴量を表示

X の分散共分散行列の固有値固有ベクトルを求めます．

In [ ]:
# X の分散共分散行列の固有値と固有ベクトルを求める
_, sval, Vt = np.linalg.svd(X, full_matrices=False)
eval = sval**2/N
print('固有値:', eval)
U = Vt

# 2つの固有ベクトルの値を表示
df_hoge = pd.DataFrame(index=df_feature.columns)
df_hoge['u1'] = U[0]
df_hoge['u2'] = U[1]
df_hoge

固有値の大きい方の二つの固有ベクトルの値は，上記のようになっています．

- u1 の方向は，正の値の項目と負の値の項目がおよそ半々となっています．正の値の項目への支出が平均より多く，負の値の項目への支出が平均より少ない県では，y1 の値が大きくなります．負の値の中では「他の主食的外食」の絶対値が大きいため，この項目の影響が大きそうです．
- u2 の方向は，全ての項目の値が負です．したがって，y2 の値は，外食全体への支出が少ない県で大きくなります．

これら2つの固有ベクトルを使って D = 13 次元のデータを 2 次元に変換してみましょう．

In [ ]:
# 2次元へ次元削減
Y = X @ U[:2, :].T
print(Y.shape)
print(Y[:5, :]) # 最初の5件の特徴量を表示

数値を見てもわかりにくいので，散布図にしてみます．

In [ ]:
# グラフを描く

kinki5 = np.array([n in range(24, 29) for n in range(N)]) # 滋賀，京都，大阪，兵庫，奈良
tohoku = np.array([n in range(1, 7) for n in range(N)]) # 青森，岩手，宮城，秋田，山形，福島

fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.scatter(Y[:, 0], Y[:, 1])
ax.scatter(Y[kinki5, 0], Y[kinki5, 1], label='kinki5')
ax.scatter(Y[tohoku, 0], Y[tohoku, 1], label='tohoku')
ax.axvline(0, linestyle='-', color='gray')
ax.axhline(0, linestyle='-', color='gray')
ax.set_xlim(-15000, 15000)
ax.set_ylim(-15000, 15000)
ax.set_aspect('equal')
ax.legend()
for n in range(N):
    plt.annotate(f'{n}', (Y[n, 0]+200, Y[n, 1]+200))
plt.show()

#### ★★★ やってみよう ★★★

上記の結果を見て，次のことを考えよう．結果を紙媒体にメモしておこう．

- 東北6県は
    - 外食に使う金額が多い方？それとも少ない方？
    - うどん，そば，和食，洋食，焼き肉などへの支出が多い方？
- 近畿5府県はどんな傾向？
